# Valuation service

You are building a valuation service.

On the input you've got 3 files containing:
* data.csv - product representation with price,currency,quantity,matching_id
* currencies.csv - currency code and ratio to PLN, eg. GBP,2.4 can be converted to PLN with procedure 1 PLN * 2.4
* matchings.csv - matching data matching_id,top_priced_count

Now, read all the data. From products with particular matching_id take those with the highest total price (price * quantity), limit data set by top_priced_count and aggregate prices.
Result save to top_products.csv with four columns: matching_id,total_price,avg_price,currency, ignored_products_count.





In [297]:
import pandas

currencies_csv_file = r'C:\Users\aw036048\Desktop\!data analyst!\Valuation service\currencies.csv'
matching_csv_file = r'C:\Users\aw036048\Desktop\!data analyst!\Valuation service\matchings.csv'
data_csv_file = r'C:\Users\aw036048\Desktop\!data analyst!\Valuation service\data.csv'

#read input files
currencies_csv_df = pandas.read_csv(currencies_csv_file, header=0)
matching_csv_df = pandas.read_csv(matching_csv_file,  header=0)
data_csv_df = pandas.read_csv(data_csv_file, header=0)

#set index for data frame
data_csv_df.set_index('matching_id', inplace=True, drop = False)
matching_csv_df.set_index('matching_id', inplace=True)
currencies_csv_df.set_index('currency', inplace=True, drop = False)



# Input data
#### data.csv - product representation with price ,currency, quantity,matching_id

In [298]:
data_csv_df.head()

,id,price,currency,quantity,matching_id
matching_id,,,,,
3,1,1000,GBP,2,3
1,2,1050,EU,1,1
1,3,2000,PLN,1,1
2,4,1750,EU,2,2
3,5,1400,EU,4,3


#### currencies.csv - currency code and ratio to PLN

In [299]:
currencies_csv_df.head()

,currency,ratio
currency,,
GBP,GBP,2.4
EU,EU,2.1
PLN,PLN,1.0


#### matchings.csv - matching data matching_id,top_priced_count

In [300]:
matching_csv_df.head()

,top_priced_count
matching_id,
1,2
2,2
3,3


# Data processing
Calculate total price for each purchase in polish zloty

In [370]:
#joined data frame with all inputs
joined_df = data_csv_df.join(matching_csv_df)
joined_df.set_index('currency', inplace=True)
joined_df = joined_df.join(currencies_csv_df)
total_price = pandas.DataFrame(joined_df.price*joined_df.quantity*joined_df.ratio, columns = ['total_price_zl'])
joined_df = pandas.concat([joined_df,total_price], axis=1)
joined_df.head()

,id,price,quantity,matching_id,top_priced_count,currency,ratio,total_price_zl
currency,,,,,,,,
EU,2,1050,1,1,2,EU,2.1,2205.0
EU,4,1750,2,2,2,EU,2.1,7350.0
EU,5,1400,4,3,3,EU,2.1,11760.0
EU,8,4000,1,3,3,EU,2.1,8400.0
GBP,9,1400,3,1,2,GBP,2.4,10080.0


Calculate counts of all product gruped by "matching_id" column

In [371]:
product_counts = joined_df.groupby('matching_id').id.count()
product_avg = joined_df.groupby('matching_id').total_price_zl.mean()
product_avg
#product_counts_df = product_counts.to_frame().reset_index()
#product_counts_df.columns = ['matching_id','product_counts']
#product_counts_df.set_index('matching_id', inplace=True)
#product_counts_df

matching_id
1     4761.666667
2    14175.000000
3     8130.000000
Name: total_price_zl, dtype: float64

Put all calculation to one data frame

In [409]:
#change index to new jooin
joined_df.reset_index(drop= True, inplace = True)
full_joined_df = joined_df.join(product_counts_df, on = 'matching_id')
full_joined_df ['ignored_products_count'] = (full_joined_df ['product_counts'] - full_joined_df ['top_priced_count'])
full_joined_df

,id,price,quantity,matching_id,top_priced_count,currency,ratio,total_price_zl,product_counts,ignored_products_count
0,2,1050,1,1,2,EU,2.1,2205.0,3,1
1,4,1750,2,2,2,EU,2.1,7350.0,2,0
2,5,1400,4,3,3,EU,2.1,11760.0,4,1
3,8,4000,1,3,3,EU,2.1,8400.0,4,1
4,9,1400,3,1,2,GBP,2.4,10080.0,3,1
5,1,1000,2,3,3,GBP,2.4,4800.0,4,1
6,7,630,5,3,3,GBP,2.4,7560.0,4,1
7,3,2000,1,1,2,PLN,1.0,2000.0,3,1
8,6,7000,3,2,2,PLN,1.0,21000.0,2,0


Limited data set by top_priced_count and aggregate prices

In [410]:
full_agg_df = pandas.DataFrame()
for unique_val in full_joined_df['matching_id'].unique():
    grupped_match_id = full_joined_df[full_joined_df['matching_id'] == unique_val].sort_values('total_price_zl', ascending = False)
    indicator = grupped_match_id.iloc[:1]['top_priced_count'].values[0]
    result = grupped_match_id.iloc[:indicator]
    full_agg_df = pandas.concat([full_agg_df,result])

full_agg_df

,id,price,quantity,matching_id,top_priced_count,currency,ratio,total_price_zl,product_counts,ignored_products_count
4,9,1400,3,1,2,GBP,2.4,10080.0,3,1
0,2,1050,1,1,2,EU,2.1,2205.0,3,1
8,6,7000,3,2,2,PLN,1.0,21000.0,2,0
1,4,1750,2,2,2,EU,2.1,7350.0,2,0
2,5,1400,4,3,3,EU,2.1,11760.0,4,1
3,8,4000,1,3,3,EU,2.1,8400.0,4,1
6,7,630,5,3,3,GBP,2.4,7560.0,4,1


Calculate avrage price of selected product by "maching_id" and limited by "top_priced_count"

In [411]:
product_avg = full_agg_df.groupby('matching_id').total_price_zl.mean()
product_avg = pandas.DataFrame(data = product_avg)
product_avg.columns = ['avg_price']
#join product_avg to full_agg_df
full_agg_df = full_agg_df.join(product_avg, on = 'matching_id')
product_avg

,avg_price
matching_id,
1,6142.5
2,14175.0
3,9240.0


# Results

In [412]:
# matching_id,total_price,avg_price,currency, ignored_products_count
full_agg_df.drop(['id','price','quantity','ratio','product_counts','top_priced_count'], axis=1 , inplace = True)
full_agg_df.reindex_axis(['matching_id','total_price_zl','avg_price','currency','ignored_products_count'
], axis = 1).reset_index(drop = True)

,matching_id,total_price_zl,avg_price,currency,ignored_products_count
0,1,10080.0,6142.5,GBP,1
1,1,2205.0,6142.5,EU,1
2,2,21000.0,14175.0,PLN,0
3,2,7350.0,14175.0,EU,0
4,3,11760.0,9240.0,EU,1
5,3,8400.0,9240.0,EU,1
6,3,7560.0,9240.0,GBP,1
